In [3]:

from absl import flags

import numpy as np
import six
import tensorflow as tf
import tensorflow_ranking as tfr

flags.DEFINE_string("train_path", "/home/sandro/repo/ranking/tensorflow_ranking/examples/data/train.txt", "Input file path used for training.")
flags.DEFINE_string("vali_path", "/home/sandro/repo/ranking/tensorflow_ranking/examples/data/vali.txt", "Input file path used for validation.")
flags.DEFINE_string("test_path", "/home/sandro/repo/ranking/tensorflow_ranking/examples/data/test.txt", "Input file path used for testing.")
flags.DEFINE_string("output_dir", "/tmp/tmp_out", "Output directory for models.")

flags.DEFINE_integer("train_batch_size", 32, "The batch size for training.")
flags.DEFINE_integer("num_train_steps", 100000, "Number of steps for training.")

flags.DEFINE_float("learning_rate", 0.01, "Learning rate for optimizer.")
flags.DEFINE_float("dropout_rate", 0.5, "The dropout rate before output layer.")
flags.DEFINE_list("hidden_layer_dims", ["256", "128", "64"],
                  "Sizes for hidden layers.")

flags.DEFINE_integer("num_features", 136, "Number of features per document.")
flags.DEFINE_integer("list_size", 100, "List size used for training.")
flags.DEFINE_integer("group_size", 1, "Group size used in score function.")

flags.DEFINE_string("loss", "pairwise_logistic_loss",
                    "The RankingLossKey for loss function.")

FLAGS = flags.FLAGS
num_features = 136

class IteratorInitializerHook(tf.estimator.SessionRunHook):
  """Hook to initialize data iterator after session is created."""

  def __init__(self):
    super(IteratorInitializerHook, self).__init__()
    self.iterator_initializer_fn = None

  def after_create_session(self, session, coord):
    """Initialize the iterator after the session has been created."""
    del coord
    self.iterator_initializer_fn(session)


def example_feature_columns():
  """Returns the example feature columns."""
  feature_names = ["{}".format(i + 1) for i in range(num_features)]
  return {
      name: tf.feature_column.numeric_column(
          name, shape=(1,), default_value=0.0) for name in feature_names
  }


def load_libsvm_data(path, list_size):
  """Returns features and labels in numpy.array."""

  def _parse_line(line):
    """Parses a single line in LibSVM format."""
    tokens = line.split("#")[0].split()
    assert len(tokens) >= 2, "Ill-formatted line: {}".format(line)
    label = float(tokens[0])
    qid = tokens[1]
    kv_pairs = [kv.split(":") for kv in tokens[2:]]
    features = {k: float(v) for (k, v) in kv_pairs}
    return qid, features, label

  tf.compat.v1.logging.info("Loading data from {}".format(path))

  # The 0-based index assigned to a query.
  qid_to_index = {}
  # The number of docs seen so far for a query.
  qid_to_ndoc = {}
  # Each feature is mapped an array with [num_queries, list_size, 1]. Label has
  # a shape of [num_queries, list_size]. We use list for each of them due to the
  # unknown number of quries.
  feature_map = {k: [] for k in example_feature_columns()}
  label_list = []
  total_docs = 0
  discarded_docs = 0
  with open(path, "rt") as f:
    for line in f:
      qid, features, label = _parse_line(line)
      if qid not in qid_to_index:
        # Create index and allocate space for a new query.
        qid_to_index[qid] = len(qid_to_index)
        qid_to_ndoc[qid] = 0
        for k in feature_map:
          feature_map[k].append(np.zeros([list_size, 1], dtype=np.float32))
        label_list.append(np.ones([list_size], dtype=np.float32) * -1.)
      total_docs += 1
      batch_idx = qid_to_index[qid]
      doc_idx = qid_to_ndoc[qid]
      qid_to_ndoc[qid] += 1
      # Keep the first 'list_size' docs only.
      if doc_idx >= list_size:
        discarded_docs += 1
        continue
      for k, v in six.iteritems(features):
        print(k, v)
        assert k in feature_map, "Key {} not found in features.".format(k)
        feature_map[k][batch_idx][doc_idx, 0] = v
      label_list[batch_idx][doc_idx] = label
    print(feature_map)
    return null
  tf.compat.v1.logging.info("Number of queries: {}".format(len(qid_to_index)))
  tf.compat.v1.logging.info(
      "Number of documents in total: {}".format(total_docs))
  tf.compat.v1.logging.info(
      "Number of documents discarded: {}".format(discarded_docs))

  # Convert everything to np.array.
  for k in feature_map:
    feature_map[k] = np.array(feature_map[k])
  return feature_map, np.array(label_list)


DuplicateFlagError: The flag 'train_path' is defined twice. First from /home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/ipykernel_launcher.py, Second from /home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/ipykernel_launcher.py.  Description from first occurrence: Input file path used for training.

In [5]:
class IteratorInitializerHook(tf.estimator.SessionRunHook):
  """Hook to initialize data iterator after session is created."""

  def __init__(self):
    super(IteratorInitializerHook, self).__init__()
    self.iterator_initializer_fn = None

  def after_create_session(self, session, coord):
    """Initialize the iterator after the session has been created."""
    del coord
    self.iterator_initializer_fn(session)


def example_feature_columns():
  """Returns the example feature columns."""
  feature_names = ["{}".format(i + 1) for i in range(num_features)]
  return {
      name: tf.feature_column.numeric_column(
          name, shape=(1,), default_value=0.0) for name in feature_names
  }


def load_libsvm_data(path, list_size):
  """Returns features and labels in numpy.array."""

  def _parse_line(line):
    """Parses a single line in LibSVM format."""
    tokens = line.split("#")[0].split()
    assert len(tokens) >= 2, "Ill-formatted line: {}".format(line)
    label = float(tokens[0])
    qid = tokens[1]
    kv_pairs = [kv.split(":") for kv in tokens[2:]]
    features = {k: float(v) for (k, v) in kv_pairs}
    return qid, features, label

  tf.compat.v1.logging.info("Loading data from {}".format(path))

  # The 0-based index assigned to a query.
  qid_to_index = {}
  # The number of docs seen so far for a query.
  qid_to_ndoc = {}
  # Each feature is mapped an array with [num_queries, list_size, 1]. Label has
  # a shape of [num_queries, list_size]. We use list for each of them due to the
  # unknown number of quries.
  feature_map = {k: [] for k in example_feature_columns()}
  label_list = []
  total_docs = 0
  discarded_docs = 0
  with open(path, "rt") as f:
    for line in f:
      qid, features, label = _parse_line(line)
      if qid not in qid_to_index:
        # Create index and allocate space for a new query.
        qid_to_index[qid] = len(qid_to_index)
        qid_to_ndoc[qid] = 0
        for k in feature_map:
          feature_map[k].append(np.zeros([list_size, 1], dtype=np.float32))
        label_list.append(np.ones([list_size], dtype=np.float32) * -1.)
      total_docs += 1
      batch_idx = qid_to_index[qid]
      doc_idx = qid_to_ndoc[qid]
      qid_to_ndoc[qid] += 1
      # Keep the first 'list_size' docs only.
      if doc_idx >= list_size:
        discarded_docs += 1
        continue
      for k, v in six.iteritems(features):
        print(k, v)
        assert k in feature_map, "Key {} not found in features.".format(k)
        feature_map[k][batch_idx][doc_idx, 0] = v
      label_list[batch_idx][doc_idx] = label

    tf.compat.v1.logging.info("Number of queries: {}".format(len(qid_to_index)))
    tf.compat.v1.logging.info("Number of documents in total: {}".format(total_docs))
    tf.compat.v1.logging.info("Number of documents discarded: {}".format(discarded_docs))

  # Convert everything to np.array.
    for k in feature_map:
        feature_map[k] = np.array(feature_map[k])
    return feature_map, np.array(label_list)


In [13]:
def _parse_line(line):
    """Parses a single line in LibSVM format."""
    tokens = line.split("#")[0].split()
    assert len(tokens) >= 2, "Ill-formatted line: {}".format(line)
    label = float(tokens[0])
    qid = tokens[1]
    kv_pairs = [kv.split(":") for kv in tokens[2:]]
    features = {k: float(v) for (k, v) in kv_pairs}
    return qid, features, label

In [38]:
num_features=136
path="/home/sandro/repo/ranking/tensorflow_ranking/examples/data/train_10.txt"
path="/tmp/train.txt"
list_size=100
# The 0-based index assigned to a query.
qid_to_index = {}
# The number of docs seen so far for a query.
qid_to_ndoc = {}
# Each feature is mapped an array with [num_queries, list_size, 1]. Label has
# a shape of [num_queries, list_size]. We use list for each of them due to the
# unknown number of quries.
feature_map = {k: [] for k in example_feature_columns()}
label_list = []
total_docs = 0
discarded_docs = 0
with open(path, "rt") as f:
    for line in f:
        qid, features, label = _parse_line(line)
        if qid not in qid_to_index:
            # Create index and allocate space for a new query.
            qid_to_index[qid] = len(qid_to_index)
            qid_to_ndoc[qid] = 0
        for k in feature_map:
            feature_map[k].append(np.zeros([list_size, 1], dtype=np.float32))
            label_list.append(np.ones([list_size], dtype=np.float32) * -1.)
        total_docs += 1
        batch_idx = qid_to_index[qid]
        doc_idx = qid_to_ndoc[qid]
        qid_to_ndoc[qid] += 1
        # Keep the first 'list_size' docs only.
        if doc_idx >= list_size:
            discarded_docs += 1
            continue
        for k, v in six.iteritems(features):
            print(k, v)
            assert k in feature_map, "Key {} not found in features.".format(k)
            feature_map[k][batch_idx][doc_idx, 0] = v
    label_list[batch_idx][doc_idx] = label
    
  # Convert everything to np.array.
    for k in feature_map:
        feature_map[k] = np.array(feature_map[k])
    label_list = np.array(label_list)


115 -0.617277
5 -0.109573
17 0.369905
68 -0.257625
125 0.570188
110 0.734393
136 -0.08864
135 -0.928225
133 -0.836065
13 -0.929646
63 -0.987756
131 -0.495995
84 -0.205543
107 0.956571
82 0.094344
69 0.111284
79 0.790958
42 -0.983709
75 -0.332035
84 0.926297
36 -0.071459
11 -0.522524
111 0.958885
98 -0.453638
62 0.350604
39 0.857645
105 -0.3425
88 0.094909
128 0.887842
57 -0.594232
48 -0.855241
135 0.210445
18 0.107854
81 0.769232
30 0.462099
13 0.876798
100 0.871419
118 -0.4949
29 -0.61869
114 -0.53267
27 0.928543
11 0.731187
80 -0.85078
72 0.967862
77 0.250605
86 -0.30268
132 0.807445
52 -0.217663
66 -0.46469
79 0.090635
39 -0.904325
133 -0.35428
45 0.394609
57 0.973102
64 0.959751
5 -0.329988
67 -0.945687
71 0.7402
132 0.644358
42 0.956237
102 -0.666758
117 0.771439
116 0.429287
10 -0.386381
51 -0.061408
31 0.376986
38 0.0025
103 0.388242
93 0.060171
26 -0.519465
94 0.18044
123 0.268493
131 0.51423
72 -0.344369
22 0.903892
107 0.994518
127 0.932992
102 -0.987355
24 0.46872
13 0.96266

85 0.941102
15 0.316017
40 -0.264149
67 -0.501494
110 -0.074535
14 -0.278054
6 -0.18416
64 -0.976619
5 0.366364
132 0.234624
81 0.418417
13 -0.507477
53 -0.705791
99 -0.949821
70 0.795917
134 -0.486162
113 -0.04908
74 -0.823221
83 -0.632227
56 0.109007
132 -0.670201
53 0.217761
130 -0.811898
95 -0.543872
39 -0.36907
76 -0.525285
46 0.33719
58 0.633635
84 -0.197096
47 0.706047
2 -0.5691
69 0.513131
119 0.496551
31 -0.901606
19 0.64029
26 0.608117
57 -0.142161
134 0.340512
127 0.376473
34 0.252504
102 0.111795
45 0.557636
75 -0.017705
27 0.242151
129 0.47386
111 -0.183758
66 -0.780493
97 -0.077546
104 0.094231
91 -0.046275
114 -0.909765
28 0.560383
73 0.519869
104 0.977231
18 -0.425136
93 0.657713
70 -0.171758
37 0.547586
6 -0.301367
10 -0.412007
21 -0.939111
132 0.666678
131 0.612867
55 -0.296524
62 0.623843
22 -0.978116
71 0.364837
23 -0.764947
11 0.38517
86 -0.467883
90 0.666969
55 -0.406666
69 0.682016
54 -0.790136
80 -0.028411
73 -0.913334
126 0.15879
63 0.961104
101 0.007783
134 -0

In [34]:
num_features=3
feature_map = {k: [] for k in example_feature_columns()}

In [33]:
feature_map

{'1': [],
 '2': [],
 '3': [],
 '4': [],
 '5': [],
 '6': [],
 '7': [],
 '8': [],
 '9': [],
 '10': [],
 '11': [],
 '12': [],
 '13': [],
 '14': [],
 '15': [],
 '16': [],
 '17': [],
 '18': [],
 '19': [],
 '20': [],
 '21': [],
 '22': [],
 '23': [],
 '24': [],
 '25': [],
 '26': [],
 '27': [],
 '28': [],
 '29': [],
 '30': [],
 '31': [],
 '32': [],
 '33': [],
 '34': [],
 '35': [],
 '36': [],
 '37': [],
 '38': [],
 '39': [],
 '40': [],
 '41': [],
 '42': [],
 '43': [],
 '44': [],
 '45': [],
 '46': [],
 '47': [],
 '48': [],
 '49': [],
 '50': [],
 '51': [],
 '52': [],
 '53': [],
 '54': [],
 '55': [],
 '56': [],
 '57': [],
 '58': [],
 '59': [],
 '60': [],
 '61': [],
 '62': [],
 '63': [],
 '64': [],
 '65': [],
 '66': [],
 '67': [],
 '68': [],
 '69': [],
 '70': [],
 '71': [],
 '72': [],
 '73': [],
 '74': [],
 '75': [],
 '76': [],
 '77': [],
 '78': [],
 '79': [],
 '80': [],
 '81': [],
 '82': [],
 '83': [],
 '84': [],
 '85': [],
 '86': [],
 '87': [],
 '88': [],
 '89': [],
 '90': [],
 '91': [],
 '92': [

In [39]:
def example_feature_columns():
  """Returns the example feature columns."""
  feature_names = ["{}".format(i) for i in range(num_features)]
  return {
      name: tf.feature_column.numeric_column(
          name, shape=(1,), default_value=0.0) for name in feature_names
  }

In [41]:
# OWN DATA
num_features=3
path="/home/sandro/repo/kdd-cup-2019/data/interim/train_svm.txt"
list_size=100
# The 0-based index assigned to a query.
qid_to_index = {}
# The number of docs seen so far for a query.
qid_to_ndoc = {}
# Each feature is mapped an array with [num_queries, list_size, 1]. Label has
# a shape of [num_queries, list_size]. We use list for each of them due to the
# unknown number of quries.
feature_map = {k: [] for k in example_feature_columns()}
label_list = []
total_docs = 0
discarded_docs = 0
with open(path, "rt") as f:
    for line in f:
        qid, features, label = _parse_line(line)
        if qid not in qid_to_index:
            # Create index and allocate space for a new query.
            qid_to_index[qid] = len(qid_to_index)
            qid_to_ndoc[qid] = 0
        for k in feature_map:
            feature_map[k].append(np.zeros([list_size, 1], dtype=np.float32))
            label_list.append(np.ones([list_size], dtype=np.float32) * -1.)
        total_docs += 1
        batch_idx = qid_to_index[qid]
        doc_idx = qid_to_ndoc[qid]
        qid_to_ndoc[qid] += 1
        # Keep the first 'list_size' docs only.
        if doc_idx >= list_size:
            discarded_docs += 1
            continue
        for k, v in six.iteritems(features):
            #print(k, v)
            assert k in feature_map, "Key {} not found in features.".format(k)
            feature_map[k][batch_idx][doc_idx, 0] = v
    label_list[batch_idx][doc_idx] = label
    
  # Convert everything to np.array.
    for k in feature_map:
        feature_map[k] = np.array(feature_map[k])
    label_list = np.array(label_list)


In [43]:
path="/home/sandro/repo/kdd-cup-2019/data/interim/train_svm.txt"
list_size=100
features, labels = load_libsvm_data(path, list_size)

{'0': array([[[53156.],
         [48112.],
         [48112.],
         [51641.],
         [13207.],
         [ 8175.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
       